In [1]:
import torch 
import torch_geometric
import numpy as np
from fuzzywuzzy import fuzz

torch.seed()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

/home/prerak/anaconda3/envs/kg/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/prerak/anaconda3/envs/kg/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using device cpu


In [2]:
from generate_graph import graph_to_ttl
G = graph_to_ttl()


In [5]:
nodes = set()
relations = set()

for stmt in G:
    nodes.add(str(stmt[0]))
    nodes.add(str(stmt[2]))
    relations.add(str(stmt[1]))

nodes = list(nodes)
relations = list(relations)

In [67]:
def fuzzsearch(w, is_node):
    w = w.strip()
    if(w.count("?") != 0):
        return w
    if is_node:
        scores = list(map(lambda x : fuzz.ratio(x, w), nodes))
        idxs = np.flip(np.argsort(scores))
        return nodes[idxs[0]]
    else:
        scores = list(map(lambda x : fuzz.ratio(x, w), relations))
        idxs = np.flip(np.argsort(scores))
        return relations[idxs[0]]

In [80]:
import re

def get_sparSQL(query):
    # regex = r"?[^,^)]+"
    # variables = re.findall(regex, query)
    # print(variables)
    regex = r"\(([^)]+)\)"
    relations = re.findall(regex, query)
    
    variable = query.split('[')[0].strip()
    
    sparSQL = f"SELECT DISTINCT {variable} \nWHERE {{\n"
    
    for i in relations:
        r = transform_relation(i)
        if(r != None):
            sparSQL += r
    sparSQL += "}"
    
    return sparSQL

def transform_relation(relation):
    if(relation.count('?') == 0):
        return None
    q = ""
    count = 0
    for term in relation.split(","):
        q += f"{fuzzsearch(term, count != 1)}"
        if(count != 2):
            q += " "
        count += 1
    q += ".\n"
    return q

year:2013


In [82]:
def get_results(query):
    q = get_sparSQL(query)
    qres = G.query(q)
    
    if(len(qres) == 0):
        print("Unable to find any answer")
        return
    for res in qres:
        print(f"Possible Answer: {str(res[0])}")
    

In [83]:
Q = '''?year1[("Elysium", "movie2year", ?year1), ("Elysium", "movie2actor", "Matt Damon"), ("Elysium", "movie2actor", "Jodie Foster"), ("Elysium", "movie2genre", "science fiction")]'''

get_results(Q)

Possible Answer: year:2013
